# Health Insurance Premium Prediction Project

## Project Overview
This project aims to predict health insurance premiums using Ordinary Least Squares (OLS) regression with comprehensive feature engineering and selection techniques including:
- Variance Inflation Factor (VIF) analysis for multicollinearity detection
- Correlation analysis for feature selection
- Feature engineering to improve model performance

## Dataset
The dataset contains health insurance information with features like age, sex, BMI, children, smoker status, region, and insurance charges.

## Methodology
1. **Data Exploration and Preprocessing**
2. **Feature Engineering**
3. **Feature Selection using Correlation and VIF**
4. **OLS Model Implementation**
5. **Model Evaluation and Diagnostics**

In [ ]:
# Import necessary libraries for data manipulation, visualization, and modeling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Load the insurance dataset
df = pd.read_csv('insurance.csv')

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nSummary Statistics:")
print(df.describe())

## Exploratory Data Analysis (EDA)

In [ ]:
# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())

# Check for duplicate records
print(f"\nNumber of duplicate records: {df.duplicated().sum()}")

# Display unique values for categorical columns
categorical_cols = ['sex', 'smoker', 'region']
for col in categorical_cols:
    print(f"\nUnique values in {col}: {df[col].unique()}")
    print(f"Value counts for {col}:")
    print(df[col].value_counts())

In [ ]:
# Create visualizations for data distribution
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Distribution of numerical variables
numerical_cols = ['age', 'bmi', 'children', 'charges']
for i, col in enumerate(numerical_cols):
    row = i // 2
    col_idx = i % 2
    axes[row, col_idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    axes[row, col_idx].set_title(f'Distribution of {col}')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('Frequency')

# Bar plots for categorical variables
axes[0, 2].bar(df['sex'].value_counts().index, df['sex'].value_counts().values)
axes[0, 2].set_title('Distribution of Sex')
axes[0, 2].set_xlabel('Sex')
axes[0, 2].set_ylabel('Count')

axes[1, 2].bar(df['smoker'].value_counts().index, df['smoker'].value_counts().values)
axes[1, 2].set_title('Distribution of Smoker')
axes[1, 2].set_xlabel('Smoker')
axes[1, 2].set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# Analyze the target variable (charges) in more detail
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(df['charges'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Insurance Charges')
plt.xlabel('Charges')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(np.log(df['charges']), bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Log(Charges)')
plt.xlabel('Log(Charges)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.boxplot(df['charges'])
plt.title('Boxplot of Insurance Charges')
plt.ylabel('Charges')

plt.tight_layout()
plt.show()

# Statistical summary of charges
print("Statistical Summary of Insurance Charges:")
print(f"Mean: ${df['charges'].mean():.2f}")
print(f"Median: ${df['charges'].median():.2f}")
print(f"Standard Deviation: ${df['charges'].std():.2f}")
print(f"Minimum: ${df['charges'].min():.2f}")
print(f"Maximum: ${df['charges'].max():.2f}")
print(f"Skewness: {df['charges'].skew():.2f}")
print(f"Kurtosis: {df['charges'].kurtosis():.2f}")

In [ ]:
# Analyze relationships between features and target variable
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Age vs Charges
axes[0, 0].scatter(df['age'], df['charges'], alpha=0.6)
axes[0, 0].set_title('Age vs Insurance Charges')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Charges')

# BMI vs Charges
axes[0, 1].scatter(df['bmi'], df['charges'], alpha=0.6)
axes[0, 1].set_title('BMI vs Insurance Charges')
axes[0, 1].set_xlabel('BMI')
axes[0, 1].set_ylabel('Charges')

# Children vs Charges
axes[0, 2].boxplot([df[df['children'] == i]['charges'].values for i in range(6)], 
                   labels=range(6))
axes[0, 2].set_title('Children vs Insurance Charges')
axes[0, 2].set_xlabel('Number of Children')
axes[0, 2].set_ylabel('Charges')

# Sex vs Charges
sns.boxplot(data=df, x='sex', y='charges', ax=axes[1, 0])
axes[1, 0].set_title('Sex vs Insurance Charges')

# Smoker vs Charges
sns.boxplot(data=df, x='smoker', y='charges', ax=axes[1, 1])
axes[1, 1].set_title('Smoker vs Insurance Charges')

# Region vs Charges
sns.boxplot(data=df, x='region', y='charges', ax=axes[1, 2])
axes[1, 2].set_title('Region vs Insurance Charges')
axes[1, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Feature Engineering

In [ ]:
# Create a copy of the dataset for feature engineering
df_engineered = df.copy()

# 1. Create BMI categories
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif bmi < 25:
        return 'Normal'
    elif bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'

df_engineered['bmi_category'] = df_engineered['bmi'].apply(categorize_bmi)

# 2. Create age groups
def categorize_age(age):
    if age < 25:
        return 'Young'
    elif age < 35:
        return 'Young Adult'
    elif age < 50:
        return 'Middle Age'
    else:
        return 'Senior'

df_engineered['age_group'] = df_engineered['age'].apply(categorize_age)

# 3. Create interaction features
df_engineered['age_bmi_interaction'] = df_engineered['age'] * df_engineered['bmi']
df_engineered['smoker_age_interaction'] = df_engineered['age'] * (df_engineered['smoker'] == 'yes').astype(int)
df_engineered['smoker_bmi_interaction'] = df_engineered['bmi'] * (df_engineered['smoker'] == 'yes').astype(int)

# 4. Create polynomial features
df_engineered['age_squared'] = df_engineered['age'] ** 2
df_engineered['bmi_squared'] = df_engineered['bmi'] ** 2

# 5. Create binary features for high-risk categories
df_engineered['high_bmi'] = (df_engineered['bmi'] >= 30).astype(int)
df_engineered['senior_citizen'] = (df_engineered['age'] >= 50).astype(int)

print("New features created:")
new_features = ['bmi_category', 'age_group', 'age_bmi_interaction', 'smoker_age_interaction', 
                'smoker_bmi_interaction', 'age_squared', 'bmi_squared', 'high_bmi', 'senior_citizen']
for feature in new_features:
    print(f"- {feature}")

print(f"\nDataset shape after feature engineering: {df_engineered.shape}")

In [ ]:
# Encode categorical variables using Label Encoding and One-Hot Encoding
from sklearn.preprocessing import LabelEncoder

# Label encode binary categorical variables
le_sex = LabelEncoder()
le_smoker = LabelEncoder()

df_engineered['sex_encoded'] = le_sex.fit_transform(df_engineered['sex'])
df_engineered['smoker_encoded'] = le_smoker.fit_transform(df_engineered['smoker'])

# One-hot encode multi-class categorical variables
region_dummies = pd.get_dummies(df_engineered['region'], prefix='region')
bmi_category_dummies = pd.get_dummies(df_engineered['bmi_category'], prefix='bmi_cat')
age_group_dummies = pd.get_dummies(df_engineered['age_group'], prefix='age_grp')

# Combine all features
df_final = pd.concat([df_engineered, region_dummies, bmi_category_dummies, age_group_dummies], axis=1)

# Drop original categorical columns
columns_to_drop = ['sex', 'smoker', 'region', 'bmi_category', 'age_group']
df_final = df_final.drop(columns=columns_to_drop)

print("Categorical encoding completed!")
print(f"Final dataset shape: {df_final.shape}")
print("\nFinal features:")
print(df_final.columns.tolist())

## Feature Selection using Correlation Analysis

In [ ]:
# Calculate correlation matrix
correlation_matrix = df_final.corr()

# Create correlation heatmap
plt.figure(figsize=(16, 12))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.5}, mask=mask)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Identify highly correlated features (correlation > 0.8 or < -0.8)
def find_high_correlation_pairs(corr_matrix, threshold=0.8):
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
    return high_corr_pairs

high_corr_pairs = find_high_correlation_pairs(correlation_matrix, threshold=0.8)
print("Highly correlated feature pairs (|correlation| > 0.8):")
for pair in high_corr_pairs:
    print(f"{pair[0]} - {pair[1]}: {pair[2]:.3f}")

# Correlation with target variable
target_correlation = correlation_matrix['charges'].sort_values(key=abs, ascending=False)
print("\nCorrelation with target variable (charges):")
print(target_correlation)

In [ ]:
# Remove highly correlated features based on correlation analysis
# We'll remove one feature from each highly correlated pair, keeping the one more correlated with target

features_to_remove = set()

# Remove features with very low correlation with target (|correlation| < 0.05)
low_correlation_features = target_correlation[abs(target_correlation) < 0.05].index.tolist()
if 'charges' in low_correlation_features:
    low_correlation_features.remove('charges')

print("Features with low correlation to target (|correlation| < 0.05):")
print(low_correlation_features)

# For highly correlated pairs, remove the one with lower target correlation
for pair in high_corr_pairs:
    feature1, feature2 = pair[0], pair[1]
    if feature1 != 'charges' and feature2 != 'charges':
        corr1 = abs(target_correlation[feature1])
        corr2 = abs(target_correlation[feature2])
        if corr1 < corr2:
            features_to_remove.add(feature1)
        else:
            features_to_remove.add(feature2)

# Combine all features to remove
features_to_remove.update(low_correlation_features)
features_to_remove = list(features_to_remove)

print(f"\nFeatures to remove based on correlation analysis: {features_to_remove}")

# Create dataset after correlation-based feature removal
df_corr_selected = df_final.drop(columns=features_to_remove)
print(f"\nDataset shape after correlation-based selection: {df_corr_selected.shape}")
print("Remaining features:")
print([col for col in df_corr_selected.columns if col != 'charges'])